In [10]:
import tensorflow as tf
'''
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)
'''

'''
session1 = tf.Session()
with tf.device('/gpu:0'):
    config = tf.ConfigProto(intra_op_parallelism_threads=32,\
           inter_op_parallelism_threads=32, allow_soft_placement=True,\
           device_count = {'CPU' : 1, 'GPU' : 1})
    session1 = tf.Session(config=config)
#K.set_session(session1)
session2 = tf.Session()
with tf.device('/gpu:1'):
    config = tf.ConfigProto(intra_op_parallelism_threads=32,\
           inter_op_parallelism_threads=32, allow_soft_placement=True,\
           device_count = {'CPU' : 1, 'GPU' : 1})
    session2 = tf.Session(config=config)
'''

config = tf.ConfigProto(
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.75)
)

self.sess = tf.Session(config=config)
K.set_session(self.sess)

NameError: name 'self' is not defined

In [11]:
#with session1.as_default():
from keras_transformer import *

import numpy as np
from keras_layer_normalization import LayerNormalization
from keras_multi_head import MultiHeadAttention
from keras_position_wise_feed_forward import FeedForward
from keras_pos_embd import TrigPosEmbedding
from keras_embed_sim import EmbeddingRet, EmbeddingSim
from keras.models import load_model

from keras.layers import Input, Lambda,RepeatVector,Dense,Reshape,Dropout
from keras.models import Model
from keras import backend as K
import keras


def get_m(token_num,
          embed_dim,
          encoder_num,
          decoder_num,
          head_num,
          hidden_dim,
          attention_activation=None,
          feed_forward_activation='relu',
          dropout_rate=0.0,
          embed_weights =None,
          embed_trainable=None,
          trainable=True,
          use_adapter=False,
          adapter_units=None,
          adapter_activation='relu'):

    decoder_token_num = token_num

    decoder_embed_weights = embed_weights

    if decoder_embed_weights is not None:
        decoder_embed_weights = [decoder_embed_weights]

    decoder_embed_trainable = embed_trainable

    if decoder_embed_trainable is None:
        decoder_embed_trainable = decoder_embed_weights is None


    decoder_embed_layer = EmbeddingRet(
        input_dim=decoder_token_num,
        output_dim=embed_dim,
        mask_zero=True,
        weights=decoder_embed_weights,
        trainable=decoder_embed_trainable,
        name='Decoder-Token-Embedding',
    )

    encoder_input = keras.layers.Input(shape=(None,100), name='Encoder-Input')
    pos_wised_encoder = TrigPosEmbedding(
        mode=TrigPosEmbedding.MODE_ADD,
        name='Encoder-Embedding',
    )(encoder_input)
    #pos_wised_encoder = Lambda(get_position_encoding, name='Encoder_With_Positions')(encoder_input)

    encoded_layer = get_encoders(
        encoder_num=encoder_num,
        input_layer=pos_wised_encoder,#encoder_input,
        head_num=head_num,
        hidden_dim=hidden_dim,
        attention_activation=attention_activation,
        feed_forward_activation=feed_forward_activation,
        dropout_rate=dropout_rate,
        trainable=trainable,
        use_adapter=use_adapter,
        adapter_units=adapter_units,
        adapter_activation=adapter_activation,
    )

    decoder_input = keras.layers.Input(shape=(None,), name='Decoder-Input') 
    decoder_embed, decoder_embed_weights = decoder_embed_layer(decoder_input)
    decoder_embed = TrigPosEmbedding(
        mode=TrigPosEmbedding.MODE_ADD,
        name='Decoder-Embedding',
    )(decoder_embed)
    decoded_layer = get_decoders(
        decoder_num=decoder_num,
        input_layer=decoder_embed,
        encoded_layer=encoded_layer,
        head_num=head_num,
        hidden_dim=hidden_dim,
        attention_activation=attention_activation,
        feed_forward_activation=feed_forward_activation,
        dropout_rate=dropout_rate,
        trainable=trainable,
        use_adapter=use_adapter,
        adapter_units=adapter_units,
        adapter_activation=adapter_activation,
    )
    dense_layer = EmbeddingSim(
        trainable=trainable,
        name='Output',
    )([decoded_layer, decoder_embed_weights])
    return keras.models.Model(inputs=[encoder_input,decoder_input], outputs=dense_layer)

In [12]:
from keras.utils import to_categorical


def piecesextender(data):
    kol= np.zeros(shape=(data.shape[0]))
    for i in range(data.shape[0]):
        kol[i]=np.count_nonzero(data[i])
    kol = kol.astype(np.int)
    out = np.zeros(shape=(np.sum(kol)-2*kol.shape[0],512))
    buf = 0
    for g in range(data.shape[0]):        
        for k in range(1,kol[g]-1):
            out[buf][0]=3000
            out[buf][1:513-k]+=data[g][k:]
            buf+=1
    return out, kol

def vectorsextender(data,kol):
    out = np.zeros(shape=(np.sum(kol)-2*data.shape[0],512))
    buf=0
    for g in range(data.shape[0]):
        for k in range(kol[g]-2):
            out[buf]+=data[g]
            buf+=1
    return out

def positionencoder(kol):
    out = np.zeros(shape=(np.sum(kol)-2*kol.shape[0],1))
    out = out.astype(np.int)
    buf=0
    for g in range(len(kol)):
        for k in range(1,kol[g]-1):
            out[buf]=k
            buf+=1
    return out

def vectors_per_words(vectors,kols):#,model):
    from keras.models import load_model
    import numpy as np

    encoder_shifter_model = load_model('D:/112/decoder/models/sen_to_word2_0.h5')
    out = np.zeros(shape=(vectors.shape[0],512,100))
    kol = kols.tolist()
    #with session2.as_default():
    for i in range(vectors.shape[0]):
        out[i][kol[i][0]] = np.ones(100)
        for k in range(kol[i][0]-1):
            out[i][k+1]=encoder_shifter_model.predict(
                [vectors[i].reshape(1,512),
                 np.array([k+1])])[1]
        out[i][kol[i][0]+2:]-= np.ones(shape=(512-kol[i][0]-2,100))
        out[i][kol[i][0]+2:]-= np.ones(shape=(512-kol[i][0]-2,100))
    np.save('D:/112/decoder/data/buf.npy', out)
    #return out 
    
def f(text):
    print(text)
    

In [13]:
#with session1.as_default():
model = get_m(
    token_num=3003,
    embed_dim=100,
    encoder_num=3,
    decoder_num=6,
    head_num=4,
    hidden_dim=120,
    attention_activation='relu',
    feed_forward_activation='relu',
    dropout_rate=0.05,
    embed_weights=np.random.random((3003, 100)),
    use_adapter=False,
)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
)
#model.save('D:/112/decoder/models/dec_buf.h5')
#model.summary()

In [4]:
#model.save('D:/112/decoder/models/dec_buf.h5')

In [15]:
from multiprocessing import Process
p = Process(target=f, args=('Bob'))
p.start()
p.join()


In [14]:
from scipy.ndimage.interpolation import shift
from multiprocessing import Process

for i in range(1000):
    vectors_train = np.load('D:/112/decoder/data/data3000/news_vectors'+str(i)+'.npy')
    pieces_train = np.load('D:/112/decoder/data/data3000/news_pieces'+str(i)+'.npy')
    shifting = np.ones(shape=(pieces_train.shape[0],512))
    pieces_train = pieces_train+shifting
    
    print('Readed')  
        
    for j in range(90):
        ptt, kol = piecesextender(pieces_train[0+j*5:(1+j)*5])
        pot = np.copy(ptt)
        pot = shift(pot, (0,-1))
        pot = pot.reshape(pot.shape[0],512,1)
        vtt = vectorsextender(vectors_train[0+j*5:(1+j)*5], kol)
        pos = positionencoder(kol)
        p = Process(target=vectors_per_words, args=(vtt,pos))
        p.start()
        p.join()
        #vpwt=vectors_per_words(vtt,pos,encoder_shifter_model)
        print('Converted') 
        
        vpwt=np.load('D:/112/decoder/data/buf.npy')
        #reset_keras()
        #with session1.as_default():
        model.train_on_batch(
            x=[vpwt,
               ptt],
            y = pot
        )
        
    ptt, kol = piecesextender(pieces_train[450:])
    pot = np.copy(ptt)
    pot = shift(pot, (0,-1))
    pot = pot.reshape(pot.shape[0],512,1)
    vtt = vectorsextender(vectors_train[450:], kol)
    pos = positionencoder(kol)        
    vpwt=vectors_per_words(vtt,kol, encoder_shifter_model)
    #with session1.as_default():
    model.test_on_batch(
        x=[vpwt,
           ptt],
        y = pot
    )    
    model.save('D:/decoder/models/m'+str(i)+'.h5')

Readed


BrokenPipeError: [Errno 32] Broken pipe

In [14]:
model.train_on_batch(
            x=[vpwt,
               ptt],
            y = pot,
            reset_metrics=False
        )

TypeError: train_on_batch() got an unexpected keyword argument 'reset_metrics'

In [2]:
path=''
print(path=='f')

False
